# General remarks and introduction



## LinAlg

Throughout the package, we will see instantiations of `LinAlg`. We originally developed the package working with `numpy`, but soon found that having `torch` as a back-end for all numerics would be very useful due to GPU support and automatic differentiation. Since the two packages have slightly different API, we decided to create a unified API which is `LinAlg`. This class also takes care of random number ceeds, device on which tensors are stored (in the case of torch) and some basic random sampling and numerical functions.

In [2]:
from sbmfi.core.linalg import LinAlg
la = LinAlg(
    backend = 'torch',  # torch or numpy backend
    batch_size = 2,  # in all labelling simulation algorithms, we simulate in batches; this parameter specifies the batch_size
    device = 'cpu',
    fkwargs = None,
    seed = 42,
)

## Models, Reactions and Metabolites

The package has been developed based on the structure of [`cobrapy`](https://cobrapy.readthedocs.io/en/latest/index.html). The intention is that anyone working with `cobra` can use `sbmfi` quickly. The general phylosophy on the relation between the two packages is that one should build a model and perform various quality-checks using `cobra` and once a model is 'finished', it can be equipped with labelling information and used to sample fluxes and simulate labelling in `sbmfi`. 

For these reasons, `sbmfi.core` has modules defining the `LabelledMetabolite(cobra.Metabolite)`, `LabellingReaction(cobra.Reaction)` and `LabellingModel(cobra.Model)` classes which inherit functionality from their respective `cobra` classes. Each of these `Labelling` classes is instantiated with their respective `cobra` counterpart. The `Labelling...` classes specify behavior that is shared between different labelling simulation algorithms. We implemented the EMU algorithm {cite}`Antoniewicz2007` through `EMU_Metabolite(LabelledMetabolite)`, `EMU_Reaction(LabellingReaction)` and `EMU_Model(LabellingModel)`.

For convenience, we built the parsing method `model_builder_from_dict` to convert `json`-like data-structures into fully functional `LabellingModel` objects. In the cell below, we define a small CBM which is first parsed to a `cobra` model and converted to an `EMU_Model`.

In [9]:
from sbmfi.core.model import model_builder_from_dict, EMU_Model
reaction_kwargs = {
    'a_in': {
        'upper_bound': 10.0, 'lower_bound': 10.0,
        'atom_map_str': '∅ --> A/abc'
    },
    'co2_out': {
        'upper_bound': 100.0,
        'atom_map_str': 'co2/a --> ∅'
    },
    'e_out': {
        'upper_bound': 100.0,
        'atom_map_str': 'E/ab --> ∅'
    },
    'cof_out': {
        'upper_bound': 100.0,
        'reaction_str': 'cof --> ∅'  # NOTE that this is not an atom_map_str but a reaction_str; 
    },
    'v1': {
        'upper_bound': 100.0,
        'atom_map_str': 'A/abc --> B/ab + D/c + cof'
    },
    'v2': {
        'upper_bound': 100.0,
        'atom_map_str': 'A/abc --> C/bc + D/a'
    },
    'v3': {
        'upper_bound': 100.0,
        'atom_map_str': 'B/ab + D/c --> E/ac + co2/b'
    },
    'v4': {
        'upper_bound': 100.0,
        'atom_map_str': 'C/ab + D/c --> E/cb + co2/a'
    },
}
metabolite_kwargs = {
    'E': {'formula': 'C2H4O2'},
    'D': {'formula': 'CH4'},
}

cobra_model = model_builder_from_dict(metabolite_kwargs=metabolite_kwargs, reaction_kwargs=reaction_kwargs)
print(f'type of model: {type(cobra_model)}')

model = EMU_Model(linalg=la, model=cobra_model)
print(f'type of model: {type(model)}')
model.add_labelling_kwargs(metabolite_kwargs=metabolite_kwargs, reaction_kwargs=reaction_kwargs)
for reaction in model.reactions:
    print(reaction, reaction.bounds, type(reaction))

type of model: <class 'cobra.core.model.Model'>
type of model: <class 'sbmfi.core.model.EMU_Model'>
a_in:  --> A/abc (10.0, 10.0) <class 'sbmfi.core.reaction.EMU_Reaction'>
co2_out: co2/a -->  (0.0, 100.0) <class 'sbmfi.core.reaction.EMU_Reaction'>
e_out: E/ab -->  (0.0, 100.0) <class 'sbmfi.core.reaction.EMU_Reaction'>
cof_out: cof -->  (0.0, 100.0) <class 'cobra.core.reaction.Reaction'>
v1: A/abc --> B/ab + D/c + cof (0.0, 100.0) <class 'sbmfi.core.reaction.EMU_Reaction'>
v2: A/abc --> C/bc + D/a (0.0, 100.0) <class 'sbmfi.core.reaction.EMU_Reaction'>
v3: B/ab + D/c --> E/ac + co2/b (0.0, 100.0) <class 'sbmfi.core.reaction.EMU_Reaction'>
v4: C/ab + D/c --> E/cb + co2/a (0.0, 100.0) <class 'sbmfi.core.reaction.EMU_Reaction'>


Now we set the $^{13}C$-labelling of the substrate metabolite, which in this case is metabolite `A`. The convention is that $0$s indicate a $^{12}C$ atom and $1$s indicate a $^{13}C$ atom

In [17]:
import pandas as pd
substrate_labelling = pd.Series({f'A/011': 1.0,}, name='input')
model.set_substrate_labelling(substrate_labelling)

model.substrate_labelling

ValueError: 

Last, we set the metabolite(-fragments) that we can measure given our experimental capabilities. This is important information for the EMU algorithm.

In [18]:
model.set_measurements(measurement_list=['E']) 

In the cell below, we build the simulator. `free_reaction_id` specifies which fluxes are considered free when using the row reduced echelon form (RREF) kernel basis and transformed coordinates.

## References

```{bibliography}
:filter: docname in docnames
:style: unsrt
```